In [2]:
# Import dependancies
import numpy as np
import pandas as pd
import os
import pickle

# Load feature definitions
with open('SWAT_feat_description.pkl', 'rb') as f:
    feat_descript = pickle.load(f)

with open('SWAT_feat_names.pkl', 'rb') as f:
    feat_names = pickle.load(f)


In [23]:
print(feat_descript['WYLD'])

Water yield (mm H2O). Total amount of water leaving the HRU and entering main channel during the time step. (WYLD = SURQ + LATQ + GWQ - TLOSS - pond abstractions)


In [2]:
# Get a list of all output filenames 
# This is for hru files 
path = '/Users/muneeza/Documents/GitHub/GNNs_PrecisionAgriculture/SWAT Data Reader/'
names_list = os.listdir(path+'region02_ML/ML_R02_hru/')



In [3]:
def feature_locations(type, n_feat):

    # Grab all features in output file, if using this setting must add code for 
    # encoding land use/cover
    if type == 'all':       
        feat_spaces = np.zeros((n_feat,2)).astype(int)
        feat_spaces[0,:] = 0, 4     # LULC
        feat_spaces[1,:] = 5, 9     # HRU
        feat_spaces[2,:] = 10, 19   # GIS
        feat_spaces[3,:] = 20, 24   # SUB
        feat_spaces[4,:] = 25, 29   # MGT
        feat_spaces[5,:] = 30, 34   # MON
        feat_spaces[6,:] = 34, 44   # AREA
        st = 7
        
    # Grab numeric features only 
    elif type == 'numeric':     
        feat_spaces = np.zeros((n_feat, 2)).astype(int)
        feat_spaces[0,:] = 30, 34       # MON
        feat_spaces[1,:] = 34, 44       # AREA
        st = 2

    for i in range(st,n_feat):
        tmp = feat_spaces[i-1,1]
        if (i == st+66 or i ==st+67):
            feat_spaces[i,:] = tmp+1 , tmp+11
        else: 
            feat_spaces[i,:] = tmp, tmp+10 
    
    return(feat_spaces)

In [11]:
#   in  : (path , filename , save_dir)
#   out : saves a numpy array of the hru data in 'save_dir' with 'filename'

for i, name in enumerate(names_list):
    if name == '020200070203_hru.hru':       # has a corrupted variable with **** values
        continue
    else: 
        with open(path+'region02_ML/ML_R02_hru/'+name) as f:
            lines = f.readlines()
        f.close

        n_feat = 81
        n_smpl = len(lines[6:])

        # Generate spacing array to read the varaiables in .hru file
        feat_spaces = feature_locations('numeric',n_feat)

        # Select feature values from hru file based on character space outlined in 
        feature_vals = np.zeros((n_smpl, n_feat))
        for i , row in enumerate(lines[6:]):
            for j in range(n_feat):
                feature_vals[i,j] = float(row[feat_spaces[j,0]:feat_spaces[j,1]])
                
        # Uncomment to save the as numpy array
        np.save(path+'/hru_region02_pyinp/'+name, feature_vals)

